### PROBLEM:
Use the example data set to answer following questions:
1. Which customers are most likely to churn?
2. What can we do to reduce churn?
3. Present your answers in a format you think is most appropriate for your stakeholders to get
your message across.
4. You can use whatever software and method you find appropriate to do your analysis, and
submit your code, notebook, markdown, github repo, or any format you used for your
analysis.
5. Any other findings in addition to above question 1 & 2 you want to share from your analysis?

#### Commentary:

The purpose of this notebook is to demonstrate how we can use the data provided by the company/client to come up with actionable insights and recommendations to address their problem of customer retention. 

There could be multiple ways to solve this problem. What is proposed below is just one of the ways. Note - the emphasis is not on machine learning algorithms or techniques but a framework to solve the data probem.

### Import required packages

In [ ]:
import pandas as pd
import numpy as np

import sys

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

plt.style.use('ggplot')

width = 3
height = 3
plt.figure(figsize=(width, height))
sns.set(style="ticks", color_codes=True)

import random

from sklearn.preprocessing import Imputer

import sklearn.model_selection as model_selection
from sklearn.model_selection import train_test_split

from featexp import get_univariate_plots
from featexp import get_trend_stats

from feature_selector import FeatureSelector

import pickle

from collections import OrderedDict
import sys

from sklearn.dummy import DummyClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, cross_val_score
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, auc, f1_score, classification_report, precision_score, recall_score

import lightgbm as lgb

random.seed(3)

from pdpbox import info_plots, get_dataset

from xgboost import plot_importance

import shap
shap.initjs() 

import warnings
warnings.filterwarnings('ignore')

# import iml

In [ ]:
sys.version

#### Commentary:
    
Bringing together and assembling the data that you see in the spreadsheet could be a massive exercise by itself. This will likely involve pulling data from different data sources (databases, tables, flat files etc.), performing data quality
checks and aggregations, if required, before consolidating it to a single dataset. Further, the analyst will have to engage with stakeholders and the business to define the target variable (i.e. the outcome of interest) based on the business problem. In this case, the target variable 'Churn' has already been defined for us, which means company losing customers or customers stopping to pay for the services. 

Customer can stop doing business with a company and switch to a different one due to multiple reasons such as price, customer service, product features etc. This is called voluntary churn. Customers could also churn for other reasons such as credit card expiry, falling behind payments etc. and this is called involunatry churn. As you can see, these are 2 different problems altogether. Therefore, it is important to understand the business problem clearly and define the target variable accordingly.

For this problem, let's make the assumption that customer is churning because of the shortcomings on the company's part - i.e. voluntary churn.

### Import data

In [ ]:
# Import the csv file
file_name = 'C:/Users/Lenovo/Desktop/churn.csv' # Provide the path of the csv file here based on where you save it
df = pd.read_csv(file_name,na_values=' ') # Attribute 'totalcharges' has blank values and hence converted as string while importing

### Inspect data

In [ ]:
# Check the shape of imported dataframe
df.shape
print ("In the imported dataframe, the number of observations is {rows} and the number of variables is {cols}".format(rows= df.shape[0],cols= df.shape[1]))

In [ ]:
# Change all columns names to lower-case for consistency and standardization
df.columns = map(str.lower, df.columns)

In [ ]:
# Take a peek at the dataframe
df.head()

#### Commentary

'customerid' is the identifier variable

'churn' is the target variable

There is a combination of categorical and continuous variables in the data

In [ ]:
# Check nulls and data types
df.info()

'total charges' seems to have very few missing values. Let's check this later. Format of all the variables look alright and as expected.

#### Commentary:
    
If the data types don't look okay, they need to be converted from chararcter to numeric or vice versa. For eg. postcode or zipcode has to be converted to character if it is read as numeric while/after importing into a dataframe and then treated or transformed based on how you want to use it

In [ ]:
# Check for duplicates
df['customerid'].nunique()
print ("The number of duplicates in the dataframe is {num_dupes}".format(num_dupes = len(df['customerid']) - df['customerid'].nunique()))

In [ ]:
# Convert target variable to binary
df.loc[df.churn=='Yes','churn'] = 1
df.loc[df.churn=='No','churn'] = 0
df['churn'] = df['churn'].astype('int')

In [ ]:
# Target variable distribution
target_dist = pd.value_counts(df['churn'],normalize=True)
print ("The churn rate in the population is {:.2f}%".format(target_dist.iloc[1]*100))

#### Commentary:

Not sure what the churn benchmarks are in the telecom industry but ~27% is high. The company seems to be facing serious challenges with customer retention as around 3 out of 10 customers are leaving!

In [ ]:
# Check missing value distribution
print ("Missing value distribution in dataframe")
df.isna().mean()*100

#### Commentary:

'totalcharges' column has 11 missing values (~ 0.15%). We will choose a suitable imputation strategy, if required, post EDA

Note - Unlike this dataset, which is clean, real life datasets are dirty and a lot of data pre-processing will likely be required to bring ito into a shape that is suitable to build a model on. This data cleansing process generally involves ~70-80% of time and effort in a data science engagement.

In [ ]:
# Get count of different datatypes
df.drop(['customerid','churn'],axis=1).dtypes.value_counts()

Majority of features in the dataframe are categorical variables

In [ ]:
# Check number of unique values for each of the variables
df.nunique()

#### Commentary

Let's convert 'seniorcitizen', which is currently binary, into categorical for EDA.

This dataset doesn't have categorical variables with high cardinality (i.e. too many categories). Look up on this topic on the web to familiarize yourself on ways to treat data with high cardinality.

In [ ]:
# Frequency distribution of seniorcitizen variable
df['seniorcitizen'].value_counts()

In [ ]:
# Convert seniorcitizen to categorical variable for purposes of EDA
df.loc[df.seniorcitizen== 1,'seniorcitizen'] = 'Yes'
df.loc[df.seniorcitizen== 0,'seniorcitizen'] = 'No'
df['seniorcitizen'].value_counts()

In [ ]:
# Assign identifier and target variables
id_col = ['customerid']
target_col = ['churn']

### Exploratory Data Analaysis (EDA) - categorical variables

In [ ]:
# Count plots for all categorical variables
# %matplotlib inline
# cat_cols = [x for x in df.select_dtypes(include=['object', 'category']).columns if x not in id_col if x not in target_col]
# for col in cat_cols:
#     sns.catplot(x=col, kind="count", data=df)
#     plt.xticks(rotation=45)
#     plt.title("\n\n\n\n Count plot for {}".format(col))
#     plt.show()

In [ ]:
# Bar plots for all categorical variables against the target variable
cat_cols = [x for x in df.select_dtypes(include=['object', 'category']).columns if x not in id_col if x not in target_col]
for col in cat_cols:
    sns.catplot(x=col, y="churn", kind="bar", data=df, ci=None)
    plt.axhline(y=0.265, ls='--', c='red')
    plt.text(0,0.27, "Average churn rate in the population")
    plt.xticks(rotation=45)
    plt.title(" \n\n\n\n Churn rate by categories of '{}'".format(col))

#### Commentary:

We are interested in the categories of variables that exhibit a higher than average churn rate, i.e. the bars in the plots that cross the dotted line, which represents the average churn rate (i.e. ~27%)

Observations:

Customer segments that have a significantly higher churn rate based on the bar plots above-

+ No partners (eg. single, separated, widow etc ...)
+ No dependents (eg. either single, married with no kids...)
+ Senior Citizens
+ Multiple lines
+ Fiber Optic
+ No online security/online backup/device protection/techsupport
+ With month-to-month subscription
+ No paperless billing (likely electronic)
+ Electronic payment

The business or telecom SME can help validate initial findings, but the initial findings look intuitive. Customers who are likely to churn are those who are/have-- 
- young or retired singles
- digitally savvy (electronic billing, payments etc.)
- shorter contract periods (monthly subscription)
- fewer features or add-ons on their products (no security, backup etc.)
- experienced issues with internet service esp. Fiber Optic

### EDA - discrete and continuous variables

In [ ]:
# Histogram plots for all discrete and continuous variables
num_cols = [x for x in df.select_dtypes(include=['int64','float']).columns if x not in id_col if x not in target_col]
for col in num_cols:
    plt.title("\n\n\n\n Distribution plot for {}".format(col))
    plt.hist(df[col], bins = 10)
    plt.show()

#### Commentary:

+ Tenure is bimodal
+ Monthly charges is skewed towards lower monthly charges
+ Total charges has a long right tail and left skewed

In [ ]:
# Box plots for all discrete and continuous variables against the target variable
num_cols = [x for x in df.select_dtypes(include=['int64','float']).columns if x not in id_col if x not in target_col]
for col in num_cols:
    sns.catplot(x=col, y="churn", kind="box", orient="h", data=df)
    plt.title("\n\n\n\n Box plot for '{}' by churn category (0/1)".format(col))
    plt.show()

#### Commentary:

+ As expected lower tenure expected with higher churn rate
+ Higher monthly charges associated with churn
+ Interesting. Customers whose total charges are lesser tend to churn


NOTE - You could further slice and dice the dataset and do a deep-dive in terms of EDA to extract additional insights.

### Missing value imputation

#### Commentary:
    
Most machine learning algorithms expect non-null values in data, so it is important that input data for modelling doesn't have any missing values (i.e by applying an appropriate strategy)
    
The proportion of missing values for 'totalcharges' is insignificant (< 1%). We can choose to exclude these customers as it will not introduce any potential bias by doing so. Let's not remove any records, instead use simple imputation strategies. 

In [ ]:
df.describe(include=np.number)

#### Commentary:

The mean and median are quite far away from each other for totalcharges as we saw in the histogram. Since the distribution is left skewed, let's impute by median which is a better representation of central measure than the mean.

In [ ]:
# Missing value imputation for totalcharges
imr = Imputer(missing_values=np.NaN, strategy="median")
imr = imr.fit(df[['totalcharges']])
df["totalcharges"] = imr.transform(df[["totalcharges"]]).ravel()

In [ ]:
# Sanity check if missing values have been imputed
df["totalcharges"].isna().sum().sum()

### Outlier detection & treatment

We will use tree-based GBMs (Gradient Boosting Machines) which are robust to outliers, instead of GLM (Generalized Linear Models), which are sensitive to extreme values in data. Although it depends on a lot of factors, without over-generalizing, GBMs are known to outperform GLMs. Hence we will not do any outlier treatment. Will revisit if required.

In [ ]:
# Cap outliers based on IQR
# def winsorise(x):
#     q1, q3 = np.percentile(x,(25,75))
#     iqr = q3 - q1
#     limit = q3 + 1.5*iqr
#     return [min(i, limit) for i in x]

# df_w = df.loc[:,['totalcharges','monthlycharges']].apply(winsorise)

### Feature engineering

#### Commentary 
Feature engineering is the process of deriving additional variables from the data that could be useful in predicting the likelihood of certain activity (eg. churn). Domain expertise and understanding of the business is vital to craft additional features. 

Note - We could employ automated feature engineering packages - featuretools but will avoid as the process is computationally expensive and also will have to regularize (drop redundant features) later if we have too many features that are hard to explain

In [ ]:
# Function to create a new feature to binarize Internet service
def f(x):
  if x['internetservice'] == 'No': return 1
  else: return 0

In [ ]:
# Apply the above function
df['feat_internet_service'] = df.apply(f, axis=1)
df['feat_internet_service'].value_counts()

# Replace values of categorical variables to avoid confusion when converted to dummies
df.replace("No internet service", "no_internet", inplace=True)
df.replace("No phone service", "no_phone",inplace=True)

In [ ]:
# Create a bunch of simple features based on certain combinations i.e service and features. These may or may not be useful
df['feat_gpd'] = df['gender'] + "_Partner_" + df['partner'] + "_Dependents_" + df['dependents'] # gender + partners + dependents
df['feat_gs'] = df['gender'] + "_Sr_Citizen_" + df['seniorcitizen']  # gender + senior citizen
df['feat_gscp'] = df['gender'] + "_Sr_Citizen_" + df['seniorcitizen'] + "_Partner_" + df['partner'] # gender + senior citizen +partner
df['feat_sb'] = "Security_" + df['onlinesecurity'] + "_Backup_" + df['onlinebackup'] # Online security + Backup
df['feat_sbd'] = "Security_" + df['onlinesecurity'] + "_Backup_" + df['onlinebackup'] + "_DeviceProtect_" + df['deviceprotection'] # Online security + Online backup + Device protection
df['feat_sbdt'] = "Security_" + df['onlinesecurity'] + "_Backup_" + df['onlinebackup'] + "_DeviceProtect_" + df['deviceprotection'] + "_TechSupport_" + df['techsupport'] # Online security + Online backup + Device protection + Tech Support
df['feat_stream'] =  "StreamingTV_" + df['streamingtv'] + "_StreamingMovies_" + df['streamingmovies'] # Streaming TV + Streaming Movies

In [ ]:
# Feature engineering for the charges variable
df['feat_charges_ratio'] = df['monthlycharges']/df['totalcharges'] # Monthly charges as ratio of total charges
df['feat_charges_delta'] = df['totalcharges'] - df['monthlycharges'] # Difference in total and monthly charges

### One hot encoding or dummification of categorical variables

#### Commentary 

Most Machine learning algorithms cannot work with categorical data so they need to be converted to numbers

In [ ]:
# Insert all categorical variables into a list
cat_cols = [x for x in df.select_dtypes(include=['object', 'category']).columns if x not in id_col if x not in target_col]

In [ ]:
# Dummify all categorical variables (create n-1 dummies for n levels of a categorical variable to avoid perfect collinearity)
df_ohe = pd.get_dummies(data=df, columns=cat_cols, drop_first=True)
# df_ohe_copy = df_ohe.copy()

### Check for target leakage and correlations (collinearity)

In [ ]:
# Correlation between target variable - Churn and the features
corr_matrix = pd.DataFrame(df_ohe[df_ohe.columns[1:]].corr()['churn'][:])
corr_matrix.drop('churn',axis=0,inplace=True)
corr_matrix.columns = ['Corr_coeff']
corr_matrix = corr_matrix.iloc[corr_matrix['Corr_coeff'].abs().argsort()]
corr_matrix.tail()

#### Commentary:

Models need to be simple. Having too many redundant features makes models suffer from overfitting (i.e. model will not be able to generalize well on unseen data) and make the model less interpretable. Also, any variable that is highly correlated to target variable should be dropped as this could be self-defining.

Good read on data/target leakage - https://www.datarobot.com/wiki/target-leakage/

Correlation coefficients suggest that the variable with the highest correlation is tenure (negative correlation) but is not strong so there is no evidence of any target leakage

In [ ]:
# Find highly correlated features
features = [x for x in df_ohe.columns if x not in id_col if x not in target_col]
df_feat_sel = df_ohe[features]

fs_coll = FeatureSelector(data=df_feat_sel, labels = features)
fs_coll.identify_collinear(correlation_threshold = 0.9)

In [ ]:
# Plot for variables that are highly collinear (>0.9)
fs_coll.plot_collinear()

In [ ]:
# List of collinear features that we will remove
collinear_features = fs_coll.ops['collinear']

# Dataframe of collinear features
fs_coll.record_collinear.head()

In [ ]:
# Make a list of redundant features to be dropped
drop_feat = fs_coll.record_collinear.drop_feature.unique().tolist()
drop_feat

#### Commentary 

Looks like a majority of new features derived as part of feature engineering are useless!

In [ ]:
# Remove redundant features from the dataframe
df_ohe.drop(drop_feat,axis=1,inplace=True)

### Feature Selection

#### Remove predictors that have zero feature importance (i.e. no impact on the target variable - churn)

In [ ]:
# Pass in the appropriate parameters
features = [x for x in df_ohe.columns if x not in id_col if x not in target_col]
df_feat_sel = df_ohe[features]

target_var = df_ohe['churn']
fs_fi = FeatureSelector(data= df_feat_sel, labels = target_var)

fs_fi.identify_zero_importance(task = 'classification', 
                            eval_metric = 'auc', 
                            n_iterations = 10, 
                             early_stopping = True)
# list of zero importance features
zero_importance_features = fs_fi.ops['zero_importance']

In [ ]:
# plot the feature importances
fs_fi.plot_feature_importances(threshold = 0.99, plot_n = 10)

#### Commentary

The feature derived feat_charges_ratio - Monthly charges as ratio of total charges turns out to be the second most important feature in predicting customer churn!

There are 55 attributes that is able to explain maximum variation in data

In [ ]:
zero_importance_features

In [ ]:
# Remove zero importance features from the dataframe
# df_ohe.drop(zero_importance_features,axis=1,inplace=True)

In [ ]:
# Identify features with low importance
fs_fi.identify_low_importance(cumulative_importance = 0.99)

#### Commentary

Let's not drop features with low importance for now although their contribution is marginal. 

You can come back to this step and check what happens if we remove these features and refit the model.

In [ ]:
# Check features with 0 variance
fs_fi.identify_single_unique()

### Univariate plots to check consistency of features when data is split into train and holdout

In [ ]:
# Prepare data for univariate plots
modelling_df = df_ohe.iloc[:,1:] 

In [ ]:
# Split data for univariate plots
data_train, data_holdout = train_test_split(modelling_df,test_size=0.2,random_state=3)

In [ ]:
# Plots for univariate analysis
get_univariate_plots(data=data_train,target_col='churn',data_test = data_holdout, bins=10)

#### Commentary

Comparison of feature trends in train and holdout

Featexp calculates two metrics to display on these plots which help with gauging noisiness:

Trend correlation(seen in test plot): If a feature doesn’t hold same trend w.r.t. target across train and evaluation sets, it can lead to overfitting. This happens because the model is learning something which is not applicable in test data. Trend correlation helps understand how similar train/test trends are and mean target values for bins in train & test are used to calculate it. 

Trend changes: Sudden and repeated changes in trend direction could imply noisiness. But, such trend change can also happen because that bin has a very different population in terms of other features and hence, its incidence rate can’t really be compared with other bins.

SOURCE - https://www.kdnuggets.com/2018/11/secret-sauce-top-kaggle-competition.html

In [ ]:
# Stats to check for correlation trends between train and test
stats = get_trend_stats(data=data_train, target_col='churn', data_test=data_holdout)

In [ ]:
# Obtain correlation trend stats
stats_low_corr = stats.loc[stats['Trend_correlation'] < 0.8,:]
stats_low_corr.head(10)

In [ ]:
# These features will be removed prior to train and holdout split as there is no correlation or a perfect positive/negative correlation
low_corr_trend_feat = stats_low_corr.Feature.to_list()

### Split data into train and holdout for model build and validation

In [ ]:
# Prepare dataframes for train and test split
features = [x for x in df_ohe.columns if x not in id_col if x not in target_col if x not in low_corr_trend_feat]
features_with_cust = [x for x in df_ohe.columns if x not in target_col if x not in low_corr_trend_feat]
X = df_ohe[features]
X_with_cust = df_ohe[features_with_cust]
y = df_ohe[target_col]
ID = df_ohe[id_col]
len(features)

In [ ]:
# Export processed data to csv
# all_col = id_col + features + target_col
# processed_df = df_ohe[all_col]
# processed_df.to_csv('C:/Users/Lenovo/Desktop/processed_df.csv',index=False)

In [ ]:
# Split data into train and holdout

X_train_cust,X_holdout_cust,y_train,y_holdout = train_test_split(X_with_cust,y,test_size=0.3)

X_train = X_train_cust.drop(id_col,axis=1).reset_index(drop=True)
X_holdout = X_holdout_cust.drop(id_col,axis=1).reset_index(drop=True)

train_cust = X_train_cust[id_col].reset_index(drop=True)
holdout_cust = X_holdout_cust[id_col].reset_index(drop=True)

y_train = y_train.reset_index(drop=True)
y_holdout = y_holdout.reset_index(drop=True)

In [ ]:
print("Target distribution in train sample")
y_train.value_counts(normalize=True)

In [ ]:
print("Target distribution in holdout sample")
y_train.value_counts(normalize=True)

#### Commentary

The distribution of target variable in train and holdout samples look similar

## Train the model

#### Commentary:

We will use XGBoost for model build, without much hyper-parameter tuning. You can experiment with any binary classification algorithm you like - Logistic Regression, Support Vector Machines, Decision trees (CHAID, CART), Random Forest, GBM variants, Neural Networks, KNN etc.

The model performance metric (eg. AUC-ROC, AUC-PR, log-loss, accuracy etc.) depends on the objective.

We will consider AUC-ROC (ability of model to rank order well) as our primary model performance metric for this problem. We are interested in the model clearly separating the churners from no-churners or in other words exhibits good discriminatory power (rank-ordering ability). AUC-ROC is an appropriate metric for this purpose.

Refer to this article for a list of model performance metrics - https://www.analyticsvidhya.com/blog/2019/08/11-important-model-evaluation-error-metrics/

In [ ]:
# XGboost with default parameters with 5 fold CV
model = XGBClassifier()
kfold = KFold(n_splits=5, random_state=3)
cv_results = cross_val_score(model,X_train,y_train, cv=kfold,scoring='roc_auc')
print ("The mean AUC-ROC on 5 fold cross validation is {AUC}".format(AUC = cv_results.mean()))

#### Commentary

The mean AUC-ROC on 5 fold cross validation is 0.84. AUC-ROC ranges from 0.5 to 1. Higher the better. Models with AUC-ROC > 0.7 are considered to be fair. Based on the CV scores, model performance looks good.

In [ ]:
# Fit the XGBoostCalssifier on train data
model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_holdout, y_holdout)], 
            early_stopping_rounds=100, eval_metric='auc', verbose=10)

In [ ]:
# Make predictions on holdout sample
predictions = model.predict(X_holdout)

### Test the model on hold-out

In [ ]:
# Compute AUC-ROC on hold-out sample
predictions_probas = model.predict_proba(X_holdout)[:,1]
print("AUC-ROC score on holdout sample is :", round(roc_auc_score(y_holdout, predictions_probas),5))

In [ ]:
# Extract predicted probabilities into a dataframe
preds = pd.DataFrame(predictions_probas,columns=['pred_proba'])

In [ ]:
# Attach customer numbers to predicted probabilities 
holdout_pred_proba_cust = pd.concat([holdout_cust,preds],axis=1)
holdout_pred_proba_cust.set_index('customerid', inplace=True)

#### Commentary

The model generalizes well on hold-out sample with AUC-ROC of 0.85 and is comparable to that of train data. No evidence of overfitting.

In [ ]:
# Plot AUC ROC
fpr, tpr, thresh = roc_curve(y_holdout, predictions_probas, pos_label=1)

random_probs = [0 for i in range(len(y_holdout))]
p_fpr, p_tpr, _ = roc_curve(y_holdout, random_probs, pos_label=1)

# plot roc curves
plt.plot(fpr, tpr, linestyle='--',color='skyblue', label='XGBoost Classifier')
plt.plot(p_fpr, p_tpr, linestyle='--', color='red', label='No model (Random)')
# title
plt.title('ROC curve')
# x label
plt.xlabel('False Positive Rate')
# y label
plt.ylabel('True Positive rate')

plt.legend(loc='best')
plt.savefig('ROC',dpi=300)
plt.show()

#### Commentary:
    
Good read on AUC-ROC and its interpretation - https://towardsdatascience.com/understanding-the-roc-and-auc-curves-a05b68550b69

In [ ]:
# Compute accuracy on holdout sample
print("Accuracy score on holdout sample is", round(accuracy_score(y_holdout, predictions),5))

In [ ]:
# Precision, Recall and F1 score on holdout sample
print(classification_report(y_holdout,predictions))

### Global Feature Importance

In [ ]:
# Feature Importance
fig, ax = plt.subplots(figsize=(5,5))
plot_importance(model, max_num_features=15, height=0.8, ax=ax)
plt.show()

### Save model

In [ ]:
# Saving the model for future use
file_name = "model_churn.pkl"

# save
pickle.dump(model, open(file_name, "wb"))

# load
# model_loaded = pickle.load(open(file_name, "rb"))

#### Commentary:
    
The analysis below will help identify the customers who the company will have to retain as they are likely to churn.

### Decile tables, Lift and Gains charts

In [ ]:
# Functions for plotting

def plot_pandas_style(styler):
    from IPython.core.display import HTML
    html = '\n'.join([line.lstrip() for line in styler.render().split('\n')])
    return HTML(html)

def highlight_max(s,color='red'):
    '''
    highlight the maximum in a Series red.
    '''
    is_max = s == s.max()
    return ['background-color: {}'.format(color) if v else '' for v in is_max]

def decile_labels(agg1,label,color='skyblue'):
    agg_dummy = pd.DataFrame(OrderedDict((('TOTAL',0),('TARGET',0),('NONTARGET',0),('PCT_TAR',0),('CUM_TAR',0),('CUM_NONTAR',0),('DIST_TAR',0),('DIST_NONTAR',0),('SPREAD',0))),index=[0])
    agg1 = agg1.append(agg_dummy).sort_index()
    agg1.index.name = label
    agg1 = agg1.style.apply(highlight_max, color = 'red', subset=['SPREAD'])
    agg1.bar(subset=['TARGET'], color='{}'.format(color))
    agg1.bar(subset=['TOTAL'], color='{}'.format(color))
    agg1.bar(subset=['PCT_TAR'], color='{}'.format(color))
    return(agg1)

def deciling(data,decile_by,target,nontarget):
    inputs = list(decile_by)
    inputs.extend((target,nontarget))
    decile = data[inputs]
    grouped = decile.groupby(decile_by)
    agg1 = pd.DataFrame({},index=[])
    agg1['TOTAL'] = grouped.sum()[nontarget] + grouped.sum()[target]
    agg1['TARGET'] = grouped.sum()[target]
    agg1['NONTARGET'] = grouped.sum()[nontarget]
    agg1['PCT_TAR'] = grouped.mean()[target]*100
    agg1['CUM_TAR'] = grouped.sum()[target].cumsum()
    agg1['CUM_NONTAR'] = grouped.sum()[nontarget].cumsum()
    agg1['DIST_TAR'] = agg1['CUM_TAR']/agg1['TARGET'].sum()*100
    agg1['DIST_NONTAR'] = agg1['CUM_NONTAR']/agg1['NONTARGET'].sum()*100
    agg1['SPREAD'] = (agg1['DIST_TAR'] - agg1['DIST_NONTAR'])
    agg1 = decile_labels(agg1,'DECILE',color='skyblue')
    return(plot_pandas_style(agg1))

In [ ]:
# Preparing data to calculate churn rate by decile
scores_holdout = pd.DataFrame(model.predict_proba(X_holdout)[:,1], columns = ['SCORE'])
scores_holdout['DECILE'] = pd.qcut(scores_holdout['SCORE'].rank(method = 'first'),10,labels=range(10,0,-1))
scores_holdout['DECILE'] = scores_holdout['DECILE'].astype(int)
scores_holdout['TARGET'] = y_holdout.values
scores_holdout['NONTARGET'] = 1 - y_holdout.values

In [ ]:
# Print decile table for holdout sample
print("Decile table for holdout sample")
deciling(scores_holdout,['DECILE'],'TARGET','NONTARGET')

#### Commentary

The table above is generated by rank-ordering customers by the predicted probabilities and splitting the sample into approx. 10 equal groups. Decision on how to implement the model is based on this table.

TOTAL = Total customers in that decile
TARGET = Churner, NON TARGET = Non-churner 
PCT_TAR = Proportion of churners in the decile
CUM_TAR = Cumulative number of churners
CUM_NON_TAR = Cumulative number of non-churners
DIST_TAR = Distribution of churners (# Churners in the decile/Total # Churners)
DIST_NONTAR = Distribution of non-churners (# Non-Churners in the decile/Total # Non-Churners)
SPREAD 

Model is rank ordering very well on holdout sample without any discontinuity. ~70% of churners are captured within the first 3 deciles. A cutoff at either the 3rd -- ~70% churners or 4th decile -- ~80% churners will be optimum depending on marketing budget, contact strategy, retention unit capacity etc. Below the 4th decile, it is a point of diminishing return. This is because the average churn rate in the population is ~27%, however the churn rate for decile 5 is less than this ~24%. 

In [ ]:
# Functions for plotting lift, PvO and gain charts
def plots(agg1,target,type):

    plt.figure(1,figsize=(20, 5))

    plt.subplot(131)
    plt.plot(agg1['DECILE'],agg1['ACTUAL'],label='Actual')
    plt.plot(agg1['DECILE'],agg1['PRED'],label='Pred')
    plt.xticks(range(10,110,10))
    plt.legend(fontsize=15)
    plt.grid(True)
    plt.title('Actual vs Predicted', fontsize=20)
    plt.xlabel("Population %",fontsize=15)
    plt.ylabel(str(target) + " " + str(type) + " %",fontsize=15)

    plt.subplot(132)
    X = agg1['DECILE'].tolist()
    X.append(0)
    Y = agg1['DIST_TAR'].tolist()
    Y.append(0)
    plt.plot(sorted(X),sorted(Y))
    plt.plot([0, 100], [0, 100],'r--')
    plt.xticks(range(0,110,10))
    plt.yticks(range(0,110,10))
    plt.grid(True)
    plt.title('Gains Chart', fontsize=20)
    plt.xlabel("Population %",fontsize=15)
    plt.ylabel(str(target) + str(" DISTRIBUTION") + " %",fontsize=15)
    plt.annotate(round(agg1[agg1['DECILE'] == 30].DIST_TAR.item(),2),xy=[30,30], 
            xytext=(25, agg1[agg1['DECILE'] == 30].DIST_TAR.item() + 5),fontsize = 13)
    plt.annotate(round(agg1[agg1['DECILE'] == 50].DIST_TAR.item(),2),xy=[50,50], 
            xytext=(45, agg1[agg1['DECILE'] == 50].DIST_TAR.item() + 5),fontsize = 13)

    plt.subplot(133)
    plt.plot(agg1['DECILE'],agg1['LIFT'])
    plt.xticks(range(10,110,10))
    plt.grid(True)
    plt.title('Lift Chart', fontsize=20)
    plt.xlabel("Population %",fontsize=15)
    plt.ylabel("Lift",fontsize=15)

    plt.tight_layout()
    
def gains(data,decile_by,target,score):
    inputs = list(decile_by)
    inputs.extend((target,score))
    decile = data[inputs]
    grouped = decile.groupby(decile_by)
    agg1 = pd.DataFrame({},index=[])
    agg1['ACTUAL'] = grouped.mean()[target]*100
    agg1['PRED'] = grouped.mean()[score]*100
    agg1['DIST_TAR'] = grouped.sum()[target].cumsum()/grouped.sum()[target].sum()*100
    agg1.index.name = 'DECILE'
    agg1 = agg1.reset_index()
    agg1['DECILE'] = agg1['DECILE']*10
    agg1['LIFT'] = agg1['DIST_TAR']/agg1['DECILE']
    plots(agg1,target,'Distribution')

In [ ]:
# Plot charts for holdout sample
lift_holdout = pd.concat([X_holdout,scores_holdout],axis=1)
print("Charts for holdout sample")
gains(lift_holdout,['DECILE'],'TARGET','SCORE')

#### Commentary 

Model seems to be under-predicting a bit in the initial deciles. This will likely improve with fine-tuning model, so we will proceed without spending too much on trying to improve performance

### Target plots

In [ ]:
# Target plots for top 5 features
fea_imp = pd.DataFrame(list(model.get_booster().get_fscore().items()), columns = ['feature','importance']).sort_values('importance',ascending= False)
top_5_feat = fea_imp.iloc[:5,0].tolist()
top_5_feat

In [ ]:
# Target plots for top 5 features
for _ in top_5_feat:
    fig, axes, summary_df = info_plots.target_plot(
    df=data_train, feature=_, feature_name=_,
    target=target_col, show_percentile=True)

#### Commentary

Following are the high risk categories exhibiting likelihood to churn

- monthlycharges -  monthly charges between 65 to 100 USD 
- feat_charges_ratio - high monthly to total charges ratio (> 0.07)
- totalcharges - annual charges less than 325 $
- contract_Two year - customers NOT on a 2 year contract
- tenure - customer tenure less than a year (< 14 months)

## Model explainability

#### Commentary 

There is a saying in the ML space. A model that is not interpretable is like a drug bottle without a label.

Machine learning models are less interpretable (black-box). Model outputs are less useful if the end-user (call centre agent/representative) doesn't have context about a lead. For the agent to drive the conversation and retain a customer (s)he needs to understand why a customer is likely to churn. This is where a technique called SHAP is very useful to make the models explainable and the outputs consumable 

https://www.kaggle.com/diegovicente/using-shap-values-for-interpretability

https://shap.readthedocs.io/en/latest/example_notebooks/general/Explainable%20AI%20with%20Shapley%20Values.html

In [ ]:
# Create object that can calcuate SHAP values
explainer = shap.TreeExplainer(model=model)

# Calculate SHAP values
shap_values = explainer.shap_values(X_holdout)

In [ ]:
# Create dataset with SHAP values for all features in model
shap_xgb_mdl = pd.DataFrame(shap_values,columns = X_holdout.columns.values)

In [ ]:
# Add customer number to the SHAP values dataframe
shap_xgb_mdl['customerid'] = holdout_cust

In [ ]:
# Insert all features into a list
vars = [x for x in shap_xgb_mdl.columns if x not in id_col]

In [ ]:
# Create intermediate dataframe by transforming (melting) and assigning ranks
intermed_1 = pd.melt(shap_xgb_mdl,id_vars = id_col, value_vars = vars, var_name ='Feature', value_name = 'SHAP_Feature')
# intermed_1['RANK'] = intermed_1['SHAP_Feature'].abs().groupby(intermed_1['cust_no']).rank(method='first',ascending=False).astype(int)
intermed_1['RANK'] = intermed_1['SHAP_Feature'].groupby(intermed_1['customerid']).rank(method='first',ascending=False).astype(int)

In [ ]:
# Create another intermediate dataframe to get the feature values
intermed_2 = pd.melt(X_holdout_cust, id_vars = id_col, value_vars = vars, var_name = 'Feature', value_name = 'Feature_Value')

In [ ]:
# Merge the intermediate dataframes created above
holdout_feat_shap_vals = intermed_1.merge(intermed_2,on=['customerid','Feature'], how='left')

#### Commentary 

The output of the previous step is long. This is transposed in the next step.

In [ ]:
# Create output dataframe with features to be passed as insights to retention unit (call centre) to make informed convsersations with customers
output = holdout_feat_shap_vals.pivot(index='customerid',columns='RANK', values = ['Feature', 'Feature_Value', 'SHAP_Feature'])
output.columns = output.columns.get_level_values(0)+'_'+['{:}'.format(x) for x in output.columns.get_level_values(1)]
suffix = ('_1','_2','_3','_4','_5') #Depending on desirable number of features
output_final = output.loc[:,output.columns.str.endswith(suffix)]

In [ ]:
# To the output above, merge the corresponding predicted churn probabilities
output_final_with_pred_proba = pd.concat([output_final,holdout_pred_proba_cust],axis=1).sort_values(by=['pred_proba'],ascending=False)

In [ ]:
# Assign bins (split into 10) based on predicted probabilities. Bin 1 - customers with highest probability to churn, 10 - least probability to churn
output_final_with_pred_proba['percentile'] = pd.qcut(output_final_with_pred_proba['pred_proba'], 10, labels=False)
output_final_with_pred_proba['bin'] = 10 - output_final_with_pred_proba['percentile']
output_final_with_pred_proba.drop(['percentile'], axis=1, inplace=True)

In [ ]:
# Export output to csv. Avoid exporting to xls as there is a risk of records getting truncated due to row/size constraints
# output_final_with_pred_proba.to_csv('C:/Users/Lenovo/Desktop/output_raw.csv',index=True)

In [ ]:
# Bring the output to a shape that can easily be consumed by agents
column_titles = ['pred_proba','bin','Feature_1','Feature_Value_1','Feature_2','Feature_Value_2','Feature_3','Feature_Value_3']
output_for_retention_unit = output_final_with_pred_proba.reindex(columns=column_titles)

In [ ]:
# Display a sample of 5 customers 
# Unless you set a seed the output will be different each time
output_for_retention_unit.sample(5)

#### Commentary 

Let's take an example in the table above with a sample of customers to explain how this output is useful for an agent. Based on the decile analysis, we know that customers in bins 1 to 4 are in the high risk category. So, the telecom client will send only those customers who fall in bins 1 to 4 as leads to the retention unit.

Customer ID 3247-ZVOUO falls in bin 1 with a predicted probability of ~0.75. Based on the SHAP values, this customer is very likely to churn because of the following reasons in that order -

1. Proportion of monthly to total charges is high (~0.1) - Price
2. Interent serviced through fiber optic (0=No,1=Yes) - Service
3. Not on a 2 year contract (0=No,1=Yes) - Loyalty

Based on these reasons, the agent can then have an informed conversation with customer and resolve any issues which can in turn help with retention (for eg. offering a discount, resolving technical issues immediately, offering a product/feature for free for a limited time period etc.).

Similarly, the same can be extended to other leads too. Following this approach hekps with personalization based on customer profile, needs etc.

In [ ]:
# Export output to csv. Avoid exporting to xls as there is a risk of records getting truncated due to row/size constraints
output_for_retention_unit.to_csv('C:/Users/Lenovo/Desktop/output_for_retention_unit.csv',index=True)

### Directional impact of features on Churn

In [ ]:
def shap_viz(df_shap,df):

    # Make a copy of the input data
    shap_v = pd.DataFrame(df_shap)
    feature_list = df.columns
    shap_v.columns = feature_list
    df_v = df.copy().reset_index().drop('index',axis=1)
    
    # Determine the correlation in order to plot with different colors
    corr_list = list()
    for i in feature_list:
        b = np.corrcoef(shap_v[i],df_v[i])[1][0]
        corr_list.append(b)
    corr_df = pd.concat([pd.Series(feature_list),pd.Series(corr_list)],axis=1).fillna(0)
    
    # Make a data frame. Column 1 is the feature, and Column 2 is the correlation coefficient
    corr_df.columns  = ['Variable','Corr']
    corr_df['Sign'] = np.where(corr_df['Corr']>0,'skyblue','red')
    
    # Plot it
    shap_abs = np.abs(shap_v)
    k=pd.DataFrame(shap_abs.mean()).reset_index()
    k.columns = ['Variable','SHAP_abs']
    k2 = k.merge(corr_df,left_on = 'Variable',right_on='Variable',how='inner')
    k2 = k2.sort_values(by='SHAP_abs',ascending = False).head(10) # Change based on number of factors you are interested in
    colorlist = k2['Sign']
    ax = k2.plot.barh(x='Variable',y='SHAP_abs',color = colorlist, figsize=(5,6),legend=False)
    ax.set_xlabel("SHAP Value (Skyblue = Positive Impact, Red = Negative Impact)")
    return k2

In [ ]:
# SHAP feature directional impact
shap_xgb_mdl_wo_cust = shap_xgb_mdl.drop(['customerid'],axis=1)
shap_viz(shap_xgb_mdl_wo_cust,X_holdout)

## Customers generally churn voluntarily due to the following reasons -
1. Price (switch)
2. Quality of service
3. Usage
4. Add-ons, features

## Whom to target -
1. Clearly customers in the top 3 deciles (depends on appetite, costs, resources, exclusions due to marketing consent etc.) identified above have a churn rate i.e. 1.5 to 3 times of the average churn rate. 
The ROI will be higher by targeting these customers.
2. The customers who tend to churn-
+ pay higher than average monthly charges, 
+ have low tenure,
+ senior citizens,
+ likely digitally savvy (evident from mode of payments and paperless billing)
+ are on short-term contract periods
+ have a fiber optic connection (hypothesis - issues with service)

## Recommendations -

1. To price sensitive customers - Offer discounts based on Customer LifeTime Value and Tenure. Also recognise loyalty.
2. Improve stickiness of customers by signing them up for long term contracts and cross sell additional services by 
bundling products/features (eg. providing online security and backup)
3. Understand what issues Fiber Optic customers are facing and resolve them on priority and follow up until issues are resolved.
Tech support seems to be poor too, so that needs to be fixed.
4. For younger, low tenure customers who generally have a tendency to switch frequently could think of offering high speed 
internet service during non-peak times (eg. gaming, binging on Netflix etc.) at no additional cost.

We could think of overlaying the model outputs with segmentation analysis to get deeper insight on customer's behaviour.


## Additional comments -
+ A one size fits all approach will likely not be effective for retention unlike a personalised approach
+ Recommend passing on insights to call centre centre team for retention to have personlized comms with customer based on 
breakdown of individual prediction for each customer based on SHAP. For eg. a customer who is price-sensitive might be offered a 
discount, while offering upgrades/additional services to a customer for whom the service and experience is important.
+ Having temporal data over a period of time will help improve behavior of customer and likely improve performance of model
by having aggregated features. This will also help with proactive retention when there are early signs of switching.
+ Some of the additional data sources worth investigating.
    - Demographics based on postal code of where customer lives (ABS)
    - Complaints data (text mining)
    - Competitor pricing
    - Usage logs